# NewYork Taxi Dynamics 

In [6]:
from IPython.display import HTML
from IPython.display import Image
Image(url = "https://media.gettyimages.com/photos/big-line-of-yellow-taxis-in-new-york-city-picture-id157480155?s=1024x1024")

# Background

In New York City, taxicabs come in two varieties: yellow and green; they are widely recognizable symbols of the city. Taxis painted yellow (medallion taxis) are able to pick up passengers anywhere in the five boroughs. Those painted apple green (street hail livery vehicles, commonly known as "boro taxis"), which began to appear in August 2013, are allowed to pick up passengers in Upper Manhattan, the Bronx, Brooklyn, Queens (excluding LaGuardia Airport and John F. Kennedy International Airport), and Staten Island. Both types have the same fare structure. Taxicabs are operated by private companies and licensed by the New York City Taxi and Limousine Commission (TLC).

Taxicab vehicles, each of which must have a medallion to operate, are driven an average of 180 miles per shift. As of March 14, 2014, there were 51,398 individuals licensed to drive medallion taxicabs. There were 13,605 taxicab medallion licenses in existence. By July 2016, that number had dropped slightly to 13,587 medallions, or 18 lower than the 2014 total. Taxi patronage has declined since 2011 due to competition from rideshare services.

## Problem

One of the biggest challenges in taxicab business is to maintain high fleet
cabs and day to day challenge of placing the Taxis. If the Taxi is mobilized at low demand area
or placing Taxis near office areas during public holidays will result in lost opportunity at high
demand place and loss in revenue.

# Abstract

The main aim is to provide an analysis of the
characteristic yellow taxicabs in New York
based on the pickup data available through
their website. From the available data, this
project looks for insights on cab allocation per
location, the cab demand data and forecasts.


Source: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

The yellow trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts.

The analysis include:
1. Demand prediction
2. Revenue prediction
3. Cab pooling

### Demand prediction
1. Response Variable: CabRequest
2. Independent variables: Zone, Service Zones, Date, Hour, Pickuptime
3. Model used: Linear regression

  
### Revenue prediction
1. Response Variable: FareAmount
2. Independent variables: Zone, Service Zones, Date, Hour, Pickuptime
3. Model used: Random forest regressor regression

### Cab pooling
Model used: K-Means clustering



## Data Cleaning

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
from pylab import rcParams
from scipy.stats import spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pandas import *

#### Loading the dataset
#### Considering only January month data

In [57]:
from pandas import *

tp = read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-01.csv', iterator=True, chunksize=500)
df1 = concat(tp, ignore_index=True) 

In [20]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55


#### Loading the lookup dataset

In [2]:
df_lookup=pd.read_csv("C:/Users/kelvi/lookup.csv", delimiter=',')
df_lookup.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


#### Merging the lookup data and cab trip data using inner join on LocationID and PULocationID

In [91]:
df_Jan=pd.merge(df_lookup,df1, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

#### Splitting the trip pickup_time into respective date, hour and time

In [92]:
df_Jan['Time'] = pd.to_datetime(df.tpep_pickup_datetime)


In [93]:
df_Jan['hour']=df_Jan.Time.dt.hour

In [94]:
df_Jan['date']=df_Jan.Time.dt.date

#### Concatenating date and hour column

In [95]:
df_Jan['PickupTime']= df_Jan['date'].map(str) +" " +df_Jan['hour'].map(str)

#### Dropping the insignificant columns

In [96]:
df_Jan.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,0.01,N,1,85.00,0.0,0.5,14.00,0.00,0.3,99.80,2018-01-01 3
1,1,Newark Airport,EWR,1,0.00,N,1,20.00,0.0,0.0,0.00,0.00,0.3,20.30,2018-01-01 4
2,1,Newark Airport,EWR,4,0.00,N,1,80.00,0.0,0.5,0.00,0.00,0.3,80.80,2018-01-01 4
3,1,Newark Airport,EWR,7,0.00,N,1,70.00,0.0,0.0,5.00,0.00,0.3,75.30,2018-01-01 4
4,1,Newark Airport,EWR,1,0.01,N,1,52.00,0.0,0.5,42.00,0.00,0.3,94.80,2018-01-01 6
5,1,Newark Airport,EWR,1,25.20,N,2,69.50,0.0,0.5,0.00,12.50,0.3,82.80,2018-01-01 7
6,1,Newark Airport,EWR,0,0.00,N,1,70.00,0.0,0.0,14.06,0.00,0.3,84.36,2018-01-01 8
7,1,Newark Airport,EWR,1,0.02,N,3,-2.50,0.0,-0.5,0.00,0.00,-0.3,-3.30,2018-01-01 8
8,1,Newark Airport,EWR,1,0.02,N,2,2.50,0.0,0.5,0.00,0.00,0.3,3.30,2018-01-01 8
9,1,Newark Airport,EWR,2,0.00,N,1,75.00,0.0,0.0,18.66,18.00,0.3,111.96,2018-01-01 9


In [69]:
#df['name_match'] = df['Zone'].apply(lambda x: 'True' if x == 'NaN')
df10 = df_Jan[df_Jan.isnull().any(axis=1)]

print (df10['LocationID'].unique())

[264 265]


In [29]:
df_Jan.head()

,LocationID,Borough,Zone,service_zone,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Time,hour,date,PickupTime
0,1,EWR,Newark Airport,EWR,2,2018-01-01 03:24:25,2018-01-01 03:24:30,1,0.01,5,...,0.0,0.5,14.0,0.0,0.3,99.8,2018-01-01 03:24:25,3,2018-01-01,2018-01-01 3
1,1,EWR,Newark Airport,EWR,2,2018-01-01 04:56:14,2018-01-01 04:56:17,1,0.00,5,...,0.0,0.0,0.0,0.0,0.3,20.3,2018-01-01 04:56:14,4,2018-01-01,2018-01-01 4
2,1,EWR,Newark Airport,EWR,2,2018-01-01 04:19:34,2018-01-01 04:19:40,4,0.00,5,...,0.0,0.5,0.0,0.0,0.3,80.8,2018-01-01 04:19:34,4,2018-01-01,2018-01-01 4
3,1,EWR,Newark Airport,EWR,2,2018-01-01 04:49:22,2018-01-01 04:49:28,7,0.00,5,...,0.0,0.0,5.0,0.0,0.3,75.3,2018-01-01 04:49:22,4,2018-01-01,2018-01-01 4
4,1,EWR,Newark Airport,EWR,2,2018-01-01 06:18:35,2018-01-01 06:20:21,1,0.01,2,...,0.0,0.5,42.0,0.0,0.3,94.8,2018-01-01 06:18:35,6,2018-01-01,2018-01-01 6


#### Creating  a new column cab request and assigning its value as 1

In [73]:

df_Jan['CabRequest']=1

In [74]:
df_Jan.head()

,LocationID,Borough,Zone,service_zone,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Time,hour,date,PickupTime,CabRequest
0,1,EWR,Newark Airport,EWR,2,2018-01-01 03:24:25,2018-01-01 03:24:30,1,0.01,5,...,0.5,14.0,0.0,0.3,99.8,2018-01-01 03:24:25,3,2018-01-01,2018-01-01 3,1
1,1,EWR,Newark Airport,EWR,2,2018-01-01 04:56:14,2018-01-01 04:56:17,1,0.00,5,...,0.0,0.0,0.0,0.3,20.3,2018-01-01 04:56:14,4,2018-01-01,2018-01-01 4,1
2,1,EWR,Newark Airport,EWR,2,2018-01-01 04:19:34,2018-01-01 04:19:40,4,0.00,5,...,0.5,0.0,0.0,0.3,80.8,2018-01-01 04:19:34,4,2018-01-01,2018-01-01 4,1
3,1,EWR,Newark Airport,EWR,2,2018-01-01 04:49:22,2018-01-01 04:49:28,7,0.00,5,...,0.0,5.0,0.0,0.3,75.3,2018-01-01 04:49:22,4,2018-01-01,2018-01-01 4,1
4,1,EWR,Newark Airport,EWR,2,2018-01-01 06:18:35,2018-01-01 06:20:21,1,0.01,2,...,0.5,42.0,0.0,0.3,94.8,2018-01-01 06:18:35,6,2018-01-01,2018-01-01 6,1


#### Grouping by zone, pickupTime and service_zone and aggregating the cab request and total amount for further analysis

In [87]:
summaries = df.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries)

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-01-01 0  Boro Zone              1         23.30   
                        2018-01-01 1  Boro Zone              1         23.80   
                        2018-01-02 10 Boro Zone              1         27.80   
                        2018-01-03 22 Boro Zone              1          9.30   
                        2018-01-03 7  Boro Zone              1         10.30   
                        2018-01-04 1  Boro Zone              1          9.30   
                        2018-01-04 7  Boro Zone              1          7.80   
                        2018-01-06 8  Boro Zone              1         11.30   
                        2018-01-10 15 Boro Zone              1         19.80   
                        2018-01-10 21 Boro Zone              1         24.80   
                        2018-01-10 9  Bo

#### Exporting the above values to an excel file

In [85]:
summaries.to_csv('Combined_Jan2018.csv')

In [86]:
summaries.head()

CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-01-01 0  Boro Zone              1          23.3   
                        2018-01-01 1  Boro Zone              1          23.8   
                        2018-01-02 10 Boro Zone              1          27.8   
                        2018-01-03 22 Boro Zone              1           9.3   
                        2018-01-03 7  Boro Zone              1          10.3   

                                                    tip_amount  
Zone                    PickupTime    service_zone              
Allerton/Pelham Gardens 2018-01-01 0  Boro Zone            0.0  
                        2018-01-01 1  Boro Zone            0.0  
                        2018-01-02 10 Boro Zone            0.0  
                        2018-01-03 22 Boro Zone            0.0  
                        2018-01-03 7  Boro Zone            0.0

### Summaries dataset has all the required variables for further prediction
#### Hence we are using the same dataset for further analysis

## Repeating the above data cleaning methodology for rest of the months

In [2]:
tp1 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-02.csv', iterator=True, chunksize=1000)
df2 = pd.concat(tp1, ignore_index=True) 

In [5]:
df_feb=pd.merge(df_lookup,df2, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [7]:
df_feb['Time'] = pd.to_datetime(df_feb.tpep_pickup_datetime)
df_feb['hour']=df_feb.Time.dt.hour
df_feb['date']=df_feb.Time.dt.date

In [8]:
df_feb['PickupTime']= df_feb['date'].map(str) +" " +df_feb['hour'].map(str)

In [9]:
df_feb.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,0.00,N,1,109.00,0.0,0.0,0.00,0.00,0.3,109.30,2018-02-01 6
1,1,Newark Airport,EWR,1,0.00,N,1,105.00,0.0,0.5,21.16,0.00,0.3,126.96,2018-02-01 7
2,1,Newark Airport,EWR,1,0.00,N,1,100.00,0.0,0.5,25.16,25.00,0.3,150.96,2018-02-01 11
3,1,Newark Airport,EWR,1,0.70,N,1,77.00,0.0,0.0,15.00,16.00,0.3,108.30,2018-02-01 12
4,1,Newark Airport,EWR,1,0.00,N,2,0.00,0.0,0.0,0.00,0.00,0.0,0.00,2018-02-01 14
5,1,Newark Airport,EWR,1,0.00,N,1,60.00,0.0,0.5,0.01,0.00,0.3,60.81,2018-02-01 14
6,1,Newark Airport,EWR,1,0.00,N,1,95.00,0.0,0.5,10.00,0.00,0.3,105.80,2018-02-01 14
7,1,Newark Airport,EWR,1,0.00,N,1,117.30,0.0,0.0,10.00,0.00,0.3,127.60,2018-02-01 16
8,1,Newark Airport,EWR,1,0.03,N,1,114.00,0.0,0.0,28.58,0.00,0.3,142.88,2018-02-01 16
9,1,Newark Airport,EWR,1,0.00,N,1,52.00,4.5,0.5,17.45,12.50,0.3,87.25,2018-02-01 16


In [11]:
df_feb['CabRequest']=1

In [14]:
summaries1 = df_feb.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries1)
summaries1.to_csv('Combined_Feb2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-02-01 20 Boro Zone              1          3.80   
                        2018-02-02 18 Boro Zone              1          4.30   
                        2018-02-02 9  Boro Zone              3          3.30   
                        2018-02-03 0  Boro Zone              1         54.30   
                        2018-02-03 6  Boro Zone              1         18.30   
                        2018-02-05 13 Boro Zone              1         12.80   
                        2018-02-05 18 Boro Zone              1         19.32   
                        2018-02-08 23 Boro Zone              1         54.12   
                        2018-02-09 8  Boro Zone              1         13.30   
                        2018-02-10 1  Boro Zone              1          8.30   
                        2018-02-11 9  Bo

In [11]:
tp2 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-03.csv', iterator=True, chunksize=1000)
df3 = pd.concat(tp2, ignore_index=True) 

In [12]:
df_March=pd.merge(df_lookup,df3, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [13]:
df_March['Time'] = pd.to_datetime(df_March.tpep_pickup_datetime)
df_March['hour']=df_March.Time.dt.hour
df_March['date']=df_March.Time.dt.date

In [14]:
df_March['PickupTime']= df_March['date'].map(str) +" " +df_March['hour'].map(str)

In [15]:
df_March.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,2,14.70,N,1,100.00,0.0,0.0,10.00,0.00,0.3,110.30,2018-03-01 5
1,1,Newark Airport,EWR,1,0.01,N,1,99.00,0.0,0.5,10.00,0.00,0.0,109.50,2018-03-01 5
2,1,Newark Airport,EWR,1,0.00,N,2,20.00,0.0,0.0,0.00,62.70,0.3,83.00,2018-03-01 7
3,1,Newark Airport,EWR,2,0.30,N,2,0.00,0.0,0.0,0.00,0.00,0.3,0.30,2018-03-01 8
4,1,Newark Airport,EWR,2,0.00,N,1,70.00,0.0,0.0,15.00,0.00,0.3,85.30,2018-03-01 9
5,1,Newark Airport,EWR,1,13.70,N,1,90.00,0.0,0.0,0.00,0.00,0.3,90.30,2018-03-01 10
6,1,Newark Airport,EWR,1,0.00,N,1,90.75,0.0,0.5,18.31,0.00,0.3,109.86,2018-03-01 12
7,1,Newark Airport,EWR,1,0.00,N,1,2.50,0.0,0.5,17.25,83.00,0.3,103.55,2018-03-01 14
8,1,Newark Airport,EWR,1,0.00,N,2,10.00,0.0,0.5,0.00,0.00,0.3,10.80,2018-03-01 14
9,1,Newark Airport,EWR,1,3.50,N,1,40.00,0.0,0.0,0.00,0.00,0.3,40.30,2018-03-01 15


In [16]:
df_March['CabRequest']=1
summaries2 = df_March.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries2)
summaries2.to_csv('Combined_March2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-03-01 10 Boro Zone              1         10.70   
                        2018-03-01 8  Boro Zone              1         12.80   
                        2018-03-02 10 Boro Zone              1         15.30   
                        2018-03-02 8  Boro Zone              1         16.30   
                        2018-03-03 10 Boro Zone              1         10.30   
                        2018-03-03 13 Boro Zone              1         13.30   
                        2018-03-03 19 Boro Zone              1         44.31   
                        2018-03-03 2  Boro Zone              1          4.80   
                        2018-03-03 4  Boro Zone              1         21.80   
                        2018-03-04 10 Boro Zone              2         17.10   
                        2018-03-04 4  Bo

In [17]:
tp3 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-04.csv', iterator=True, chunksize=1000)
df4 = pd.concat(tp3, ignore_index=True) 

In [18]:
df_April=pd.merge(df_lookup,df4, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [19]:
df_April['Time'] = pd.to_datetime(df_April.tpep_pickup_datetime)
df_April['hour']=df_April.Time.dt.hour
df_April['date']=df_April.Time.dt.date

In [20]:
df_April['PickupTime']= df_April['date'].map(str) +" " +df_April['hour'].map(str)

In [21]:
df_April.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,0.30,N,1,85.00,0.0,0.0,15.00,0.00,0.3,100.30,2018-04-01 6
1,1,Newark Airport,EWR,1,0.00,N,1,85.30,0.0,0.0,13.40,0.00,0.3,99.00,2018-04-01 6
2,1,Newark Airport,EWR,4,0.00,N,1,80.00,0.0,0.0,5.00,0.00,0.3,85.30,2018-04-01 7
3,1,Newark Airport,EWR,4,0.00,N,1,90.00,0.0,0.0,9.00,0.00,0.3,99.30,2018-04-01 8
4,1,Newark Airport,EWR,2,0.00,N,1,97.00,0.0,0.0,5.00,0.00,0.3,102.30,2018-04-01 12
5,1,Newark Airport,EWR,2,0.00,N,1,76.00,0.0,0.5,0.00,19.30,0.3,96.10,2018-04-01 14
6,1,Newark Airport,EWR,1,0.00,N,1,75.00,0.0,0.0,0.00,20.00,0.3,95.30,2018-04-01 14
7,1,Newark Airport,EWR,1,0.00,N,1,99.00,0.0,0.0,10.00,0.00,0.3,109.30,2018-04-01 14
8,1,Newark Airport,EWR,2,0.00,N,1,95.00,0.0,0.5,23.96,24.00,0.3,143.76,2018-04-01 15
9,1,Newark Airport,EWR,2,0.00,N,1,20.00,0.0,0.5,4.16,0.00,0.3,24.96,2018-04-01 15


In [22]:
df_April['CabRequest']=1
summaries3 = df_April.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries3)
summaries3.to_csv('Combined_April2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-04-01 11 Boro Zone              1         41.80   
                        2018-04-01 2  Boro Zone              1         23.80   
                        2018-04-01 8  Boro Zone              1         44.80   
                        2018-04-01 9  Boro Zone              2         69.60   
                        2018-04-03 12 Boro Zone              1         11.30   
                        2018-04-03 13 Boro Zone              1         29.80   
                        2018-04-04 9  Boro Zone              1         47.22   
                        2018-04-05 10 Boro Zone              1          9.80   
                        2018-04-05 13 Boro Zone              1         51.36   
                        2018-04-05 3  Boro Zone              1          3.80   
                        2018-04-05 9  Bo

In [2]:
tp4 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-05.csv', iterator=True, chunksize=1000)
df5 = pd.concat(tp4, ignore_index=True) 

In [4]:
df_May=pd.merge(df_lookup,df5, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [5]:
df_May['Time'] = pd.to_datetime(df_May.tpep_pickup_datetime)
df_May['hour']=df_May.Time.dt.hour
df_May['date']=df_May.Time.dt.date

In [6]:
df_May['PickupTime']= df_May['date'].map(str) +" " +df_May['hour'].map(str)

In [7]:
df_May.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,0.00,N,1,121.50,0.0,0.0,24.35,0.00,0.3,146.15,2018-05-01 5
1,1,Newark Airport,EWR,1,0.00,N,1,70.00,0.0,0.0,0.00,15.00,0.3,85.30,2018-05-01 5
2,1,Newark Airport,EWR,1,0.00,N,2,0.00,0.0,0.0,0.00,0.00,0.0,0.00,2018-05-01 6
3,1,Newark Airport,EWR,5,5.16,N,1,19.00,0.0,0.5,3.96,0.00,0.3,23.76,2018-05-01 7
4,1,Newark Airport,EWR,1,0.00,N,1,77.00,0.0,0.0,18.66,16.00,0.3,111.96,2018-05-01 12
5,1,Newark Airport,EWR,1,0.00,N,1,80.00,0.0,0.0,16.05,0.00,0.3,96.35,2018-05-01 14
6,1,Newark Airport,EWR,9,0.00,N,1,95.00,0.0,0.5,0.00,0.00,0.3,95.80,2018-05-01 14
7,1,Newark Airport,EWR,0,0.00,N,1,95.00,0.0,0.0,20.00,0.00,0.3,115.30,2018-05-01 14
8,1,Newark Airport,EWR,2,0.00,N,1,20.00,0.0,0.0,0.00,0.00,0.3,20.30,2018-05-01 14
9,1,Newark Airport,EWR,1,0.00,N,1,91.00,0.0,0.0,0.00,0.00,0.3,91.30,2018-05-01 15


In [9]:
df_May['CabRequest']=1
summaries4 = df_May.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries4)
summaries4.to_csv('Combined_May2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-05-01 15 Boro Zone              1         32.30   
                        2018-05-01 7  Boro Zone              1         66.63   
                        2018-05-02 12 Boro Zone              1         15.30   
                        2018-05-02 7  Boro Zone              1         59.94   
                        2018-05-02 9  Boro Zone              1         15.87   
                        2018-05-03 10 Boro Zone              1         82.06   
                        2018-05-03 14 Boro Zone              1         13.80   
                        2018-05-03 7  Boro Zone              1          3.30   
                        2018-05-03 9  Boro Zone              1         11.80   
                        2018-05-04 10 Boro Zone              1         16.80   
                        2018-05-04 15 Bo

In [5]:
tp5 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-06.csv', iterator=True, chunksize=1000)
df6= pd.concat(tp5, ignore_index=True) 

In [6]:
df_June=pd.merge(df_lookup,df6, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [7]:
df_June['Time'] = pd.to_datetime(df_June.tpep_pickup_datetime)
df_June['hour']= df_June.Time.dt.hour
df_June['date']= df_June.Time.dt.date

In [8]:
df_June['PickupTime']= df_June['date'].map(str) +" " +df_June['hour'].map(str)

In [9]:
df_June.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,0.80,N,2,170.00,0.0,0.0,0.00,0.00,0.3,170.30,2018-06-01 1
1,1,Newark Airport,EWR,1,0.00,N,1,89.00,0.0,0.0,17.85,0.00,0.3,107.15,2018-06-01 4
2,1,Newark Airport,EWR,1,0.00,N,1,3.00,0.0,0.5,110.00,0.00,0.3,113.80,2018-06-01 8
3,1,Newark Airport,EWR,0,0.00,N,1,110.00,0.0,0.0,22.05,0.00,0.3,132.35,2018-06-01 8
4,1,Newark Airport,EWR,1,0.00,N,1,85.00,0.0,0.0,24.45,12.50,0.3,122.25,2018-06-01 9
5,1,Newark Airport,EWR,1,4.30,N,1,88.00,0.0,0.0,20.65,15.00,0.3,123.95,2018-06-01 9
6,1,Newark Airport,EWR,3,0.00,N,1,20.00,0.0,0.5,5.20,0.00,0.3,26.00,2018-06-01 11
7,1,Newark Airport,EWR,1,0.00,N,1,85.00,0.0,0.0,2.00,10.50,0.3,97.80,2018-06-01 12
8,1,Newark Airport,EWR,2,0.00,N,1,100.00,0.0,0.5,20.16,0.00,0.3,120.96,2018-06-01 12
9,1,Newark Airport,EWR,1,0.00,N,1,20.00,0.0,0.0,0.00,0.00,0.3,20.30,2018-06-01 13


In [10]:
df_June['CabRequest']=1
summaries5 = df_June.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries5)
summaries5.to_csv('Combined_June2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-06-01 14 Boro Zone              1         17.02   
                        2018-06-01 2  Boro Zone              1         13.30   
                        2018-06-01 6  Boro Zone              1         49.56   
                        2018-06-01 8  Boro Zone              1         27.30   
                        2018-06-02 22 Boro Zone              1         15.30   
                        2018-06-03 14 Boro Zone              1         20.80   
                        2018-06-03 19 Boro Zone              1         17.80   
                        2018-06-03 23 Boro Zone              1         54.06   
                        2018-06-03 9  Boro Zone              3         74.57   
                        2018-06-04 21 Boro Zone              1         25.80   
                        2018-06-05 15 Bo

In [11]:
tp6 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-07.csv', iterator=True, chunksize=1000)
df7= pd.concat(tp6, ignore_index=True) 

In [12]:
df_July=pd.merge(df_lookup,df7, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [13]:
df_July['Time'] = pd.to_datetime(df_July.tpep_pickup_datetime)
df_July['hour']= df_July.Time.dt.hour
df_July['date']= df_July.Time.dt.date

In [14]:
df_July['PickupTime']= df_July['date'].map(str) +" " +df_July['hour'].map(str)

In [15]:
df_July.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,0.00,N,1,80.00,0.0,0.0,0.00,0.00,0.3,80.30,2018-07-01 3
1,1,Newark Airport,EWR,1,0.00,N,2,107.00,0.0,0.0,0.00,0.00,0.3,107.30,2018-07-01 6
2,1,Newark Airport,EWR,1,17.60,N,1,85.00,0.0,0.0,2.00,0.00,0.3,87.30,2018-07-01 7
3,1,Newark Airport,EWR,1,0.00,N,1,52.00,0.0,0.5,80.00,0.00,0.3,132.80,2018-07-01 8
4,1,Newark Airport,EWR,1,0.00,N,1,80.00,0.0,0.0,0.00,0.00,0.3,80.30,2018-07-01 10
5,1,Newark Airport,EWR,1,0.00,N,1,95.00,0.0,0.0,0.00,0.00,0.3,95.30,2018-07-01 9
6,1,Newark Airport,EWR,1,0.00,N,1,98.00,0.0,0.5,19.76,0.00,0.3,118.56,2018-07-01 10
7,1,Newark Airport,EWR,1,0.00,N,1,85.00,0.0,0.5,17.16,0.00,0.3,102.96,2018-07-01 10
8,1,Newark Airport,EWR,2,0.07,N,1,96.00,0.0,0.0,10.00,18.00,0.3,124.30,2018-07-01 11
9,1,Newark Airport,EWR,1,0.00,N,1,95.00,0.0,0.5,10.00,0.00,0.3,105.80,2018-07-01 12


In [16]:
df_July['CabRequest']=1
summaries6 = df_July.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries6)
summaries6.to_csv('Combined_July2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-07-01 12 Boro Zone              2         94.76   
                        2018-07-01 14 Boro Zone              1         22.30   
                        2018-07-01 23 Boro Zone              1          4.30   
                        2018-07-02 14 Boro Zone              1         29.30   
                        2018-07-02 15 Boro Zone              1         12.80   
                        2018-07-03 12 Boro Zone              1          9.80   
                        2018-07-03 15 Boro Zone              1         14.72   
                        2018-07-03 7  Boro Zone              1         46.56   
                        2018-07-04 18 Boro Zone              1         66.56   
                        2018-07-04 19 Boro Zone              1         63.56   
                        2018-07-05 7  Bo

In [17]:
tp7 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-08.csv', iterator=True, chunksize=1000)
df8= pd.concat(tp7, ignore_index=True) 

In [18]:
df_August=pd.merge(df_lookup,df8, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [19]:
df_August['Time'] = pd.to_datetime(df_August.tpep_pickup_datetime)
df_August['hour']= df_August.Time.dt.hour
df_August['date']= df_August.Time.dt.date

In [20]:
df_August['PickupTime']= df_August['date'].map(str) +" " +df_August['hour'].map(str)

In [22]:
df_August.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime,CabRequest
0,1,Newark Airport,EWR,1,0.00,N,1,88.00,0.0,0.5,17.76,0.0,0.3,106.56,2018-08-01 0,1
1,1,Newark Airport,EWR,3,0.00,N,2,101.00,0.0,0.5,0.00,0.0,0.3,101.80,2018-08-01 5,1
2,1,Newark Airport,EWR,1,0.01,N,1,80.00,0.0,0.5,8.00,10.5,0.3,99.30,2018-08-01 4,1
3,1,Newark Airport,EWR,1,2.30,N,1,108.50,0.0,0.0,21.75,0.0,0.3,130.55,2018-08-01 5,1
4,1,Newark Airport,EWR,5,0.00,N,1,85.00,0.0,0.0,17.06,0.0,0.3,102.36,2018-08-01 5,1
5,1,Newark Airport,EWR,2,0.00,N,1,81.00,0.0,0.0,16.25,0.0,0.3,97.55,2018-08-01 6,1
6,1,Newark Airport,EWR,2,0.00,N,1,80.00,0.0,0.0,16.05,0.0,0.3,96.35,2018-08-01 6,1
7,1,Newark Airport,EWR,1,0.00,N,2,2.50,0.0,0.5,0.00,0.0,0.3,3.30,2018-08-01 7,1
8,1,Newark Airport,EWR,3,0.00,N,1,89.00,0.0,0.0,17.85,0.0,0.3,107.15,2018-08-01 10,1
9,1,Newark Airport,EWR,1,0.00,N,1,108.00,0.0,0.5,21.76,0.0,0.3,130.56,2018-08-01 11,1


In [23]:
df_August['CabRequest']=1
summaries7 = df_August.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries7)
summaries7.to_csv('Combined_August2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-08-01 10 Boro Zone              1         22.80   
                        2018-08-01 9  Boro Zone              2         17.60   
                        2018-08-02 5  Boro Zone              1         40.06   
                        2018-08-02 7  Boro Zone              1         44.06   
                        2018-08-04 11 Boro Zone              1         11.30   
                        2018-08-04 16 Boro Zone              1         50.80   
                        2018-08-04 6  Boro Zone              1         10.80   
                        2018-08-05 17 Boro Zone              1         15.30   
                        2018-08-05 21 Boro Zone              1         23.80   
                        2018-08-05 22 Boro Zone              2          0.00   
                        2018-08-05 5  Bo

In [3]:
tp8 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-09.csv', iterator=True, chunksize=1000)
df9= pd.concat(tp8, ignore_index=True) 

In [4]:
df_September=pd.merge(df_lookup,df9, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [5]:
df_September['Time'] = pd.to_datetime(df_September.tpep_pickup_datetime)
df_September['hour']= df_September.Time.dt.hour
df_September['date']= df_September.Time.dt.date

In [6]:
df_September['PickupTime']= df_September['date'].map(str) +" " +df_September['hour'].map(str)

In [7]:
df_September.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,4,0.00,N,1,21.30,0.0,0.5,4.42,0.00,0.3,26.52,2018-09-01 4
1,1,Newark Airport,EWR,1,0.03,N,1,95.00,0.0,0.5,19.16,0.00,0.3,114.96,2018-09-01 6
2,1,Newark Airport,EWR,2,0.30,N,2,0.00,0.0,0.0,0.00,0.00,0.3,0.30,2018-09-01 9
3,1,Newark Airport,EWR,2,0.30,N,1,80.00,0.0,0.0,16.05,0.00,0.3,96.35,2018-09-01 9
4,1,Newark Airport,EWR,1,17.80,N,2,20.00,0.0,0.0,0.00,0.00,0.3,20.30,2018-09-01 9
5,1,Newark Airport,EWR,1,0.70,N,1,85.00,0.0,0.0,10.00,16.50,0.3,111.80,2018-09-01 9
6,1,Newark Airport,EWR,9,0.00,N,1,92.00,0.0,0.0,18.46,0.00,0.3,110.76,2018-09-01 11
7,1,Newark Airport,EWR,2,0.00,N,1,78.00,0.0,0.5,18.26,12.50,0.3,109.56,2018-09-01 14
8,1,Newark Airport,EWR,1,3.70,N,1,55.00,0.0,0.0,0.00,0.00,0.3,55.30,2018-09-01 14
9,1,Newark Airport,EWR,4,0.00,N,1,72.00,0.0,0.0,5.55,0.00,0.3,77.85,2018-09-01 15


In [8]:
df_September['CabRequest']=1
summaries8 = df_September.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries8)
summaries8.to_csv('Combined_September2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-09-01 4  Boro Zone              1         16.30   
                        2018-09-01 5  Boro Zone              1         26.30   
                        2018-09-02 16 Boro Zone              1         15.80   
                        2018-09-02 21 Boro Zone              1         13.57   
                        2018-09-03 23 Boro Zone              1         55.30   
                        2018-09-04 13 Boro Zone              1         14.30   
                        2018-09-05 10 Boro Zone              1         67.56   
                        2018-09-05 11 Boro Zone              1         69.56   
                        2018-09-05 13 Boro Zone              1         36.56   
                        2018-09-05 14 Boro Zone              1          5.80   
                        2018-09-05 15 Bo

In [14]:
tp9 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-10.csv', iterator=True, chunksize=1000)
df10= pd.concat(tp9, ignore_index=True) 

In [10]:
df_Octuber=pd.merge(df_lookup,df10, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [11]:
df_Octuber['Time'] = pd.to_datetime(df_Octuber.tpep_pickup_datetime)
df_Octuber['hour']= df_Octuber.Time.dt.hour
df_Octuber['date']= df_Octuber.Time.dt.date
df_Octuber['PickupTime']= df_Octuber['date'].map(str) +" " +df_Octuber['hour'].map(str)

In [12]:
df_Octuber.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,4,0.00,N,1,66.00,0.0,0.0,0.00,0.00,0.3,66.30,2018-10-01 4
1,1,Newark Airport,EWR,2,0.06,N,1,150.00,0.0,0.0,34.11,20.26,0.3,204.67,2018-10-01 5
2,1,Newark Airport,EWR,1,0.00,N,1,121.00,0.0,0.5,0.00,0.00,0.3,121.80,2018-10-01 5
3,1,Newark Airport,EWR,1,0.00,N,2,0.00,0.0,0.0,0.00,0.00,0.3,0.30,2018-10-01 6
4,1,Newark Airport,EWR,2,0.00,N,1,97.00,0.0,0.5,22.06,12.50,0.3,132.36,2018-10-01 7
5,1,Newark Airport,EWR,1,0.00,N,1,70.00,0.0,0.0,2.00,0.00,0.3,72.30,2018-10-01 8
6,1,Newark Airport,EWR,1,0.15,N,1,100.00,0.0,0.0,25.08,0.00,0.3,125.38,2018-10-01 8
7,1,Newark Airport,EWR,2,0.00,N,1,75.00,0.0,0.0,10.00,17.50,0.3,102.80,2018-10-01 9
8,1,Newark Airport,EWR,1,0.00,N,1,115.30,0.0,0.5,23.22,0.00,0.3,139.32,2018-10-01 9
9,1,Newark Airport,EWR,1,14.60,N,1,2.50,0.0,0.5,23.40,90.36,0.3,117.06,2018-10-01 10


In [13]:
df_Octuber['CabRequest']=1
summaries9 = df_Octuber.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries9)
summaries9.to_csv('Combined_Octuber2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-10-01 1  Boro Zone              1         13.80   
                        2018-10-01 10 Boro Zone              1         45.56   
                        2018-10-01 11 Boro Zone              1         18.30   
                        2018-10-01 17 Boro Zone              1          8.97   
                        2018-10-01 21 Boro Zone              1         41.88   
                        2018-10-02 11 Boro Zone              1         35.54   
                        2018-10-02 14 Boro Zone              1         23.80   
                        2018-10-02 15 Boro Zone              2         77.24   
                        2018-10-02 21 Boro Zone              1         14.80   
                        2018-10-02 8  Boro Zone              1         13.57   
                        2018-10-03 10 Bo

In [18]:
tp10 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-11.csv', iterator=True, chunksize=1000)
df11= pd.concat(tp10, ignore_index=True) 

In [16]:
df_November=pd.merge(df_lookup,df11, how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [20]:
df_November['Time'] = pd.to_datetime(df_November.tpep_pickup_datetime)
df_November['hour']= df_November.Time.dt.hour
df_November['date']= df_November.Time.dt.date
df_November['PickupTime']= df_November['date'].map(str) +" " +df_November['hour'].map(str)

In [21]:
df_November.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,1.31,N,1,10.00,0.5,0.5,2.83,0.00,0.3,14.13,2018-11-01 0
1,1,Newark Airport,EWR,3,0.00,N,1,92.00,0.0,0.5,11.00,10.50,0.3,114.30,2018-11-01 4
2,1,Newark Airport,EWR,9,0.23,N,1,96.00,0.0,0.5,19.36,0.00,0.3,116.16,2018-11-01 4
3,1,Newark Airport,EWR,2,0.00,N,1,84.80,0.0,0.0,0.00,0.00,0.3,85.10,2018-11-01 6
4,1,Newark Airport,EWR,1,0.00,N,1,108.00,0.0,0.5,21.76,0.00,0.3,130.56,2018-11-01 12
5,1,Newark Airport,EWR,1,0.00,N,1,52.00,0.0,0.5,48.00,0.00,0.3,100.80,2018-11-01 13
6,1,Newark Airport,EWR,1,16.40,N,1,90.00,0.0,0.0,5.00,0.00,0.3,95.30,2018-11-01 13
7,1,Newark Airport,EWR,1,0.00,N,1,2.50,0.0,0.5,13.00,0.00,0.3,16.30,2018-11-01 14
8,1,Newark Airport,EWR,1,0.00,N,1,79.00,0.0,0.0,19.56,18.50,0.3,117.36,2018-11-01 14
9,1,Newark Airport,EWR,3,0.00,N,2,100.00,0.0,0.5,0.00,0.00,0.3,100.80,2018-11-01 15


In [22]:
df_November['CabRequest']=1
summaries10 = df_November.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries10)
summaries10.to_csv('Combined_November2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-11-01 10 Boro Zone              1          9.30   
                        2018-11-01 12 Boro Zone              1         20.30   
                        2018-11-01 13 Boro Zone              1         13.30   
                        2018-11-01 14 Boro Zone              1         53.97   
                        2018-11-01 7  Boro Zone              2        120.12   
                        2018-11-01 8  Boro Zone              1         13.00   
                        2018-11-01 9  Boro Zone              2        135.00   
                        2018-11-02 11 Boro Zone              1         36.30   
                        2018-11-02 14 Boro Zone              1         28.30   
                        2018-11-02 15 Boro Zone              2         76.74   
                        2018-11-02 18 Bo

In [3]:
tp11 = pd.read_csv('C:/Users/kelvi/taxi/yellow_tripdata_2018-12.csv', iterator=True, chunksize=1000)
df12= pd.concat(tp11, ignore_index=True) 

In [4]:
df_December=pd.merge(df_lookup,df12,how='inner', left_on=['LocationID'], right_on=['PULocationID'])

In [5]:
df_December['Time'] = pd.to_datetime(df_December.tpep_pickup_datetime)
df_December['hour']= df_December.Time.dt.hour
df_December['date']= df_December.Time.dt.date
df_December['PickupTime']= df_December['date'].map(str) +" " +df_December['hour'].map(str)

In [6]:
df_December.drop(['date','hour','Time','VendorID','RatecodeID','Borough','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID'] ,axis='columns')

,LocationID,Zone,service_zone,passenger_count,trip_distance,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupTime
0,1,Newark Airport,EWR,1,0.07,N,1,98.0,0.0,0.5,19.76,0.00,0.3,118.56,2018-12-01 6
1,1,Newark Airport,EWR,1,0.00,N,1,85.0,0.0,0.0,17.05,0.00,0.3,102.35,2018-12-01 8
2,1,Newark Airport,EWR,1,0.00,N,1,96.0,0.0,0.0,15.00,0.00,0.3,111.30,2018-12-01 10
3,1,Newark Airport,EWR,2,0.00,N,1,75.0,0.0,0.0,18.45,17.00,0.3,110.75,2018-12-01 13
4,1,Newark Airport,EWR,1,0.00,N,1,110.0,0.0,0.0,22.06,0.00,0.3,132.36,2018-12-01 14
5,1,Newark Airport,EWR,0,9.40,N,1,98.0,0.0,0.0,22.15,12.50,0.3,132.95,2018-12-01 15
6,1,Newark Airport,EWR,2,0.00,N,1,90.0,0.0,0.0,0.00,0.00,0.3,90.30,2018-12-01 14
7,1,Newark Airport,EWR,6,0.00,N,3,-100.0,0.0,0.0,0.00,-31.32,-0.3,-131.62,2018-12-01 16
8,1,Newark Airport,EWR,6,0.00,N,2,100.0,0.0,0.0,0.00,31.32,0.3,131.62,2018-12-01 16
9,1,Newark Airport,EWR,1,0.06,N,2,52.0,0.0,0.5,0.00,0.00,0.3,52.80,2018-12-01 17


In [7]:
df_December['CabRequest']=1
summaries11 = df_December.groupby(['Zone','PickupTime','service_zone']).agg({'CabRequest': 'sum',
                                 'total_amount': 'sum',
                                 'tip_amount': 'sum'})
print(summaries11)
summaries11.to_csv('Combined_December2018.csv')

                                                    CabRequest  total_amount  \
Zone                    PickupTime    service_zone                             
Allerton/Pelham Gardens 2018-12-01 10 Boro Zone              1         21.80   
                        2018-12-01 3  Boro Zone              1         78.27   
                        2018-12-02 12 Boro Zone              1          7.82   
                        2018-12-02 13 Boro Zone              1         38.13   
                        2018-12-02 14 Boro Zone              1         21.75   
                        2018-12-02 3  Boro Zone              1         80.57   
                        2018-12-02 8  Boro Zone              1         23.31   
                        2018-12-03 10 Boro Zone              1          8.30   
                        2018-12-03 11 Boro Zone              2         28.60   
                        2018-12-03 13 Boro Zone              1         12.30   
                        2018-12-03 15 Bo

# Summary

A dataset having the trip data of NewYork yellow taxis was chosen as a part of this project. The dataset has 12 excel sheets having data of each month's trip data. The dataset was divided into chunks as the dataset is very big. Each chunk was being read at a time. The lookup dataset has the data regarding the service_zone and zones where the cabs are active. Each month's dataset was merged with lookup dataset. A merged dsataset for each month is being considered . The merged dataset of each month is again merged to a single dataset and a final merged dataset is used for further analysis